In [92]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector, init_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [98]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
# cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_x101_64x4d_fpn_mstrain_3x_coco.py')
cfg = Config.fromfile('./configs/detr/detr_r50_8x2_150e_coco.py')

# cfg = './configs/detectors/detectors_htc_r101_20e_coco.py'
# ckp = "detectors_htc_r101_20e_coco_20210419_203638-348d533b.pth"

root='../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train.json' # train json 정보
# cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
# cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/detr_r50_8x2_150e_coco2'


# cfg.model.roi_head.bbox_head.num_classes = 10


# cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)

AttributeError: 'ConfigDict' object has no attribute 'validate'

In [99]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.42s)
creating index...
index created!


In [100]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4883, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3965  | 1 [Paper]     | 6352  | 2 [Paper pack]  | 897   | 3 [Metal]   | 936   | 4 [Glass]    | 982   |
| 5 [Plastic]       | 2943  | 6 [Styrofoam] | 1263  | 7 [Plastic bag] | 5178  | 8 [Battery] | 159   | 9 [Clothing] | 468   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [96]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load


2021-09-29 10:37:27,485 - mmdet - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2021-09-29 10:37:27,486 - mmcv - INFO - load model from: torchvision://resnet50
2021-09-29 10:37:27,486 - mmcv - INFO - Use load_from_torchvision loader
2021-09-29 10:37:27,696 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2021-09-29 10:37:27,917 - mmdet - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-29 10:37:27,918 - mmdet - INFO - 
backbone.bn1.weight - torch.Size([64]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-29 10:37:27,918 - mmdet - INFO - 
backbone.bn1.bias - torch.Size([64]): 
PretrainedInit: load from torchvision://resnet50 
 
2021-09-29 10:37:27,919 - mmdet - INFO - 
backbone.layer1.0.conv1.weight - torch.Size([64, 64, 1, 1]): 
PretrainedInit: load from torchvision:/

NameError: name 'load_checkpoint' is not defined

In [97]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


FileNotFoundError: CocoDataset: [Errno 2] No such file or directory: 'data/coco/annotations/instances_val2017.json'